In [3]:
import numpy as np
import pandas as pd
import tensorboard
from rdkit import Chem

import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, Timer

from lion_pytorch import Lion

if torch.cuda.is_available():
    print("cuda", torch.cuda.is_available())
    print(torch.cuda.get_device_name(0))
    torch.cuda.empty_cache()
else:
    print("CUDA is not available.")


import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pytorch_lightning.trainer.connectors.data_connector")
warnings.filterwarnings("ignore", category=UserWarning, module="lightning_fabric.plugins.environments.slurm")


torch.cuda.empty_cache()

from utils.train import MoleculeModel, MoleculeDataModule, GATv2Model, evaluate_model
from utils.prepare import MoleculeData, MoleculeDataset, FeaturizationParameters

cuda True
NVIDIA GeForce RTX 3080


In [4]:
molecule_dataset = torch.load("../data/QM_137k.pt")

In [5]:
molecule_dataset[0]

Data(x=[31, 133], edge_index=[2, 64], edge_attr=[64, 14], y=[31], smiles='CNC(=S)N/N=C/c1c(O)ccc2ccccc12')

In [6]:
batch_size = 128   
num_workers = 8  

data_module = MoleculeDataModule(molecule_dataset, batch_size=batch_size, num_workers=num_workers)

In [7]:
in_features = molecule_dataset[0].x.shape[1]
hidden_features = [64, 64]  # Размеры предобработки для каждого слоя
postprocess_hidden_features = [128, 64]  # Размеры слоёв постобработки
num_heads = [8, 8]  # Количество голов внимания для каждого слоя GATv2

edge_attr_dim = molecule_dataset[0].edge_attr.shape[1]

dropout_rates = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  


activation_fns = [nn.ReLU, nn.ReLU, nn.ReLU, nn.ReLU, nn.ReLU, nn.ReLU]

activation_fns = [nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU, nn.PReLU]

use_batch_norm = [True, True, False, False, True, True]

optimizer_class = Lion
learning_rate = 8.5e-4
weight_decay = 2e-4

step_size = 50
gamma = 0.1

max_epochs = 100
patience = 10

torch.set_float32_matmul_precision('high')

base_model = GATv2Model(
    atom_in_features=in_features,
    edge_in_features=edge_attr_dim,
    num_preprocess_layers=len(hidden_features),
    preprocess_hidden_features=hidden_features,
    num_heads=num_heads,
    dropout_rates=dropout_rates,
    activation_fns=activation_fns,
    use_batch_norm=use_batch_norm,
    num_postprocess_layers=len(postprocess_hidden_features),
    postprocess_hidden_features=postprocess_hidden_features,
    out_features=1
)

model = MoleculeModel(
    base_model=base_model,
    optimizer_class=optimizer_class,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    step_size=step_size,
    batch_size=batch_size,
    gamma=gamma,
    metric='rmse'
)

print("Model:\n", model)

checkpoint_callback = ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1, verbose=True)
early_stop_callback = EarlyStopping(monitor='val_loss', patience=patience, verbose=True, mode='min')
timer = Timer()
logger = pl.loggers.TensorBoardLogger('logs', name='GATv2')

trainer = pl.Trainer(
    max_epochs=max_epochs,
    enable_checkpointing=False,
    callbacks=[early_stop_callback, timer],
    enable_progress_bar=False,
    logger=logger,
    accelerator='gpu',
    devices=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Model:
 MoleculeModel(
  (base_model): GATv2Model(
    (atom_preprocess_layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=133, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Dropout(p=0.0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Dropout(p=0.0, inplace=False)
      )
    )
    (edge_preprocess_layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=14, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Dropout(p=0.0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features

In [8]:
trainer.fit(model, data_module)

Missing logger folder: logs/GATv2
2024-03-24 15:34:50.499536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | base_model | GATv2Model | 752 K 
------------------------------------------
752 K     Trainable params
0         Non-trainable params
752 K     Total params
3.009     Total estimated model params size (MB)
Metric val_loss improved. New best score: 0.047
Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.045
Metric val_loss improved by 0.003 >= min_delta = 0.0. New best score: 0.042
Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.042
Metric val_loss improved by 0.000 >= min_delt

In [9]:
seconds = timer.time_elapsed()
h, m, s = int(seconds // 3600), int((seconds % 3600) // 60), int(seconds % 60)

print(f"Время обучения: {h}:{m:02d}:{s:02d}")


Время обучения: 0:05:36


In [10]:
evaluate_model(model, data_module)

Test RMSE: 0.0393
Test R²: 0.9645


In [11]:
def draw_molecule(smiles, predictions):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    predictions_rounded = np.round(predictions, 2)

    for atom, pred in zip(mol.GetAtoms(), predictions_rounded):
        atom.SetProp('atomNote', str(pred))

    img = Chem.Draw.MolToImage(mol, size=(600, 600), kekulize=True)
    img.show()

#smiles = df_results.iloc[0]['smiles']
#predictions = df_results.iloc[0]['predictions']

#draw_molecule(smiles, predictions)
